In [1]:
import torch
import torchaudio
from transformers import Wav2Vec2ForCTC, Wav2Vec2Tokenizer, AutoModel, AutoFeatureExtractor
from transformers import AutoFeatureExtractor, ASTFeatureExtractor, ASTForAudioClassification, ASTModel
from IPython.display import Audio, display
import librosa
import soundfile as sf
import os
import faiss

In [3]:
# Load the Wav2Vec2.0 tokenizer and model
model = AutoModel.from_pretrained('facebook/wav2vec2-base-960h')
feature_extractor = AutoFeatureExtractor.from_pretrained("facebook/wav2vec2-base-960h")

feature_extractor_ast = AutoFeatureExtractor.from_pretrained('MIT/ast-finetuned-audioset-10-10-0.4593')
model_ast = ASTModel.from_pretrained('MIT/ast-finetuned-audioset-10-10-0.4593')

Some weights of the model checkpoint at facebook/wav2vec2-base-960h were not used when initializing Wav2Vec2Model: ['lm_head.bias', 'lm_head.weight']
- This IS expected if you are initializing Wav2Vec2Model from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing Wav2Vec2Model from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of the model checkpoint at MIT/ast-finetuned-audioset-10-10-0.4593 were not used when initializing ASTModel: ['classifier.dense.bias', 'classifier.layernorm.bias', 'classifier.layernorm.weight', 'classifier.dense.weight']
- This IS expected if you are initializing ASTModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing

In [4]:
model.to('cuda:0')
model_ast.to('cuda:0')

ASTModel(
  (embeddings): ASTEmbeddings(
    (patch_embeddings): ASTPatchEmbeddings(
      (projection): Conv2d(1, 768, kernel_size=(16, 16), stride=(10, 10))
    )
    (dropout): Dropout(p=0.0, inplace=False)
  )
  (encoder): ASTEncoder(
    (layer): ModuleList(
      (0-11): 12 x ASTLayer(
        (attention): ASTAttention(
          (attention): ASTSelfAttention(
            (query): Linear(in_features=768, out_features=768, bias=True)
            (key): Linear(in_features=768, out_features=768, bias=True)
            (value): Linear(in_features=768, out_features=768, bias=True)
            (dropout): Dropout(p=0.0, inplace=False)
          )
          (output): ASTSelfOutput(
            (dense): Linear(in_features=768, out_features=768, bias=True)
            (dropout): Dropout(p=0.0, inplace=False)
          )
        )
        (intermediate): ASTIntermediate(
          (dense): Linear(in_features=768, out_features=3072, bias=True)
          (intermediate_act_fn): GELUActivation(

## Resampling the songs to 16000 sample rate and splitting the track into 10 second small tracks

In [2]:
sample_rate = 16000
number_of_seconds = 10
dir_name = 'wav_files'
dest_dir = 'wav_files/10_seconds_samples/'

In [ ]:

for files in os.listdir(dir_name):
    if '.wav' in files:
        file_path = os.path.join(dir_name,files)
        x, sr = librosa.load(file_path, sr=16000)
        print("Splitting file {} with sample rate {}".format(files, sr))
        initial_seconds = number_of_seconds
        for i in range(0, len(x), number_of_seconds * sr):
            y = x[i: number_of_seconds * sr + (i)]
            out_file = str(initial_seconds) + "_" + files
            initial_seconds = initial_seconds + number_of_seconds
            sf.write(os.path.join(dest_dir,out_file), y, sr)
            


In [ ]:
x, sr = librosa.load('wav_files/10_seconds_samples/1000_KW20 - Various Artists.mp3.wav', sr=16000)

In [ ]:
sr

In [ ]:
waveform, sample_rate = torchaudio.load('wav_files/10_seconds_samples/1000_KW20 - Various Artists.mp3.wav')
waveform = waveform.squeeze().numpy()

print(sample_rate)

In [ ]:
display(Audio(waveform, rate=sample_rate))
print()

## Embeddings generator

In [ ]:
input_values = feature_extractor_ast(waveform, return_tensors="pt", sampling_rate=sample_rate).to('cuda:0')

In [ ]:
with torch.no_grad():
    embeddings = model_ast(**input_values).last_hidden_state
    embeddings = embeddings.mean(dim=1)
    print(embeddings.shape)

In [ ]:
embeddings.shape[1]

In [ ]:
embeddings.to('cpu').detach().numpy()

## Generate embeddings for all the small portions of the track and store it in FAISS - Wav2Vec2 and AST

In [35]:
index_wav_vec = faiss.IndexFlatL2(embeddings.shape[1])
index_ast = faiss.IndexFlatL2(embeddings.shape[1])

In [36]:
index_wav_vec.ntotal

0

In [37]:
index_ast.ntotal

0

In [44]:
index_id = 0
faiss_index_lookup_samples = {}
embeddings_ast = []
embeddings_wav = []

In [45]:
for files in os.listdir(dest_dir):
    try:
        waveform, sample_rate = torchaudio.load(os.path.join(dest_dir,files))
        print("Processing files {} with sample rate {}".format(files, sample_rate))
        waveform = waveform.squeeze().numpy()
        
        #wav2vec2
        input_values = feature_extractor(waveform, return_tensors="pt", sampling_rate=16000).to('cuda:0')
        embeddings = model(**input_values).last_hidden_state
        embeddings = embeddings.mean(dim=1).to('cpu').detach().numpy()
        embeddings_wav.append(embeddings[0])
        index_wav_vec.add(embeddings)
        
        #ast
        input_values = feature_extractor_ast(waveform, return_tensors="pt", sampling_rate=16000).to('cuda:0')
        embeddings = model_ast(**input_values).last_hidden_state
        embeddings = embeddings.mean(dim=1).to('cpu').detach().numpy()
        embeddings_ast.append(embeddings[0])
        index_ast.add(embeddings)
        
        # creating index lookup
        faiss_index_lookup_samples[index_id] = os.path.join(dest_dir,files)
        index_id = index_id + 1
    except Exception as e:
        print("Error files {}".format(files))
        print(e)

Processing files 260_Trinidad Dreams.mp3.wav with sample rate 16000
Processing files 390_Redondo & Ferreck Dawn - Tattoo Girl (OXIA Remix) - Material.mp3.wav with sample rate 16000
Processing files 20_Trinidad Dreams.mp3.wav with sample rate 16000
Processing files 390_Dalfie - Cant Think Right Now [PETS103].mp3.wav with sample rate 16000
Processing files 30_Martin Eyerer & Markus Homm - The Beach (Leena Music) #tupremiere.mp3.wav with sample rate 16000
Processing files 240_Night Glitches.mp3.wav with sample rate 16000
Processing files 420_Kiko & Spencer K - Rondo (OXIA Remix) - Clarisse Records.mp3.wav with sample rate 16000
Processing files 70_Pauli Pocket, Daniel Jaeger - Microsonic (Original Mix) [Snippet Preview].mp3.wav with sample rate 16000
Processing files 3120_Mix 09:2013.mp3.mp3.wav with sample rate 16000
Processing files 500_ASWR014 Hannes Bieger Poem For The Planet (feat. Ursula Rucker).mp3.wav with sample rate 16000
Processing files 180_A1. Jon Hester - Inertia.mp3.wav wit

Processing files 80_B2. Jon Hester - Neurons.mp3.wav with sample rate 16000
Processing files 20_Rodriguez Jr. - Nausicâa.mp3.wav with sample rate 16000
Processing files 1660_Mix 09:2013.mp3.mp3.wav with sample rate 16000
Processing files 560_Dave Seaman - Virgo Ryzin (Joeski Remix) [Selador] #tupremiere.mp3.wav with sample rate 16000
Processing files 240_Jon Hester - Flex.mp3.wav with sample rate 16000
Processing files 370_A Pinch Of Spice.mp3.wav with sample rate 16000
Processing files 3020_Mix 09:2013.mp3.mp3.wav with sample rate 16000
Processing files 290_Ben Böhmer - After Earth.mp3.wav with sample rate 16000
Processing files 380_Proelium [Awen Tales].mp3.wav with sample rate 16000
Processing files 170_Ben Böhmer - Purple Line.mp3.wav with sample rate 16000
Processing files 10_different.mp3.wav with sample rate 16000
Processing files 410_Hollt & Lake Avalon - Perceive (Extended Mix).mp3.wav with sample rate 16000
Processing files 410_Oxia & Nicolas Masseyeff - You Know (Original

Processing files 90_Delilah (pull me out of this).mp3.wav with sample rate 16000
Processing files 90_Rocket (Radio-Edit).mp3.wav with sample rate 16000
Processing files 190_Serge Devant - Life Trap.mp3.wav with sample rate 16000
Processing files 210_Jon Hester - Part 4.mp3.wav with sample rate 16000
Processing files 2150_KW20 - Various Artists.mp3.wav with sample rate 16000
Processing files 110_Jamie XX - Let’s Do It Again (Marcelo Pantani Edit).mp3.wav with sample rate 16000
Processing files 290_Feel & Frondorf - Equality.mp3.wav with sample rate 16000
Processing files 30_Analog People In A Digital World - Rose Rouge (Vocal Edit).mp3.wav with sample rate 16000
Processing files 310_Emanuel Satie feat. Nanghiti - Fade [Crosstown Rebels].mp3.wav with sample rate 16000
Processing files 460_Antic Soul - Apnea (Jon Hester Remix) [ECO011 | Premiere].mp3.wav with sample rate 16000
Processing files 410_Sasha - Trigonometry (La Fleur Remix).mp3.wav with sample rate 16000
Processing files 310_Be

Processing files 10_Kölsch feat. Abrahamsen - All That Matters (andhim rmx).mp3.wav with sample rate 16000
Processing files 440_Markus Homm & Benny Grauer - Plychord.mp3.wav with sample rate 16000
Processing files 180_Facing The Sun (Revive - Extended Mix).mp3.mp3.wav with sample rate 16000
Processing files 310_The Glitz - Woman feat. Fadila (Kevin Knapp Remix).mp3.wav with sample rate 16000
Processing files 360_Barber - Phase 2 Face.mp3.wav with sample rate 16000
Processing files 200_Huxley - Weirdest Geen.mp3.wav with sample rate 16000
Processing files 130_Second Change - Diversions Music 09.mp3.wav with sample rate 16000
Processing files 620_Mix 09:2013.mp3.mp3.wav with sample rate 16000
Processing files 290_Emil Berliner - Meuterei.mp3.wav with sample rate 16000
Processing files 480_ASWR014 Hannes Bieger Poem For The Planet (feat. Ursula Rucker).mp3.wav with sample rate 16000
Processing files 60_Gemini.mp3.wav with sample rate 16000
Processing files 280_Sanchä - Carpe (Original M

Processing files 350_Audiojack - First Flight.mp3.wav with sample rate 16000
Processing files 260_Palinodia - Spinning Voices (Ten System Remix).mp3.wav with sample rate 16000
Processing files 290_Ben Böhmer - Flug & Fall.mp3.wav with sample rate 16000
Processing files 330_B2. Jon Hester - Neurons.mp3.wav with sample rate 16000
Processing files 210_Olivier Giacomotto & J Diesel - Pure Fiction (Original Mix).mp3.wav with sample rate 16000
Processing files 30_Oliver Schories - Bengalo.mp3.wav with sample rate 16000
Processing files 210_Audiojack - Higher.mp3.wav with sample rate 16000
Processing files 280_Inxec - Lost ft Lauren Amour (Original Mix) [Of Unsound Mind] [MI4L.com].mp3.wav with sample rate 16000
Processing files 10_Trinidad Dreams.mp3.wav with sample rate 16000
Processing files 70_Joe Red - Couldnt Find A Better Name (Luigi Rocca Remix) Snippet.mp3.wav with sample rate 16000
Processing files 150_Olivier Giacomotto & J Diesel - Pure Fiction (Original Mix).mp3.wav with sample 

...

Processing files 90_Jon Hester - Sending Signals.mp3.wav with sample rate 16000
Processing files 80_Alexander Aurel - You Are the Wow (Original Mix).mp3.wav with sample rate 16000
Processing files 10_Ben Böhmer - After Earth.mp3.wav with sample rate 16000
Processing files 20_Kölsch - Loreley - Original Mix.mp3.wav with sample rate 16000
Processing files 390_GHEIST - Blue Petrel | Exploited.mp3.wav with sample rate 16000
Processing files 400_Ben Böhmer & Wood - Velvet Rebellion.mp3.wav with sample rate 16000
Processing files 130_Jon Hester - Lights Down.mp3.wav with sample rate 16000
Processing files 170_Al Usher - Lullaby For Robert (Bogdan Irkük Remix).mp3.wav with sample rate 16000
Processing files 130_Jaime XX - All Under One Roof Raving (Brown V. Bear Remix).mp3.wav with sample rate 16000
Processing files 380_CamelPhat, Cari Golden - Temperament Of The Beat.mp3.wav with sample rate 16000
Processing files 170_Kölsch feat. Abrahamsen - All That Matters (andhim rmx).mp3.wav with 

Processing files 370_Domino (Rework) - Sapiens 02.mp3.wav with sample rate 16000
Processing files 200_Gdansk.mp3.wav with sample rate 16000
Processing files 150_Venen.mp3.wav with sample rate 16000
Processing files 70_Renato Cohen - Duke Of Weird [Sincopat 67].mp3.wav with sample rate 16000
Processing files 160_Rodriguez Jr. - Nausicâa.mp3.wav with sample rate 16000
Processing files 70_1. Ben Böhmer - Der Blender.mp3.wav with sample rate 16000
Processing files 70_Jon Hester - Shouts In The Dark - DVS1 Lost Mix.mp3.wav with sample rate 16000
Processing files 1600_KW20 - Various Artists.mp3.wav with sample rate 16000
Processing files 270_CV Premiere I Jon Hester - Sustain.mp3.wav with sample rate 16000
Processing files 30_Its Time.mp3.wav with sample rate 16000
Processing files 110_Anek - Come Out To Play (Sante Remix).mp3.wav with sample rate 16000
Processing files 370_Jon Hester - Instant.mp3.wav with sample rate 16000
Processing files 60_Pele & Nico Stojan - Shifting Down (Kindisch)

Processing files 380_Lee Burridge & Matthew Dekay - Out Of Order (Original mix ).mp3.wav with sample rate 16000
Processing files 490_GHEIST - Blue Petrel | Exploited.mp3.wav with sample rate 16000
Processing files 160_Ben Böhmer - Autumn.mp3.wav with sample rate 16000
Processing files 310_Markus Homm & Benny Grauer - Plychord.mp3.wav with sample rate 16000
Processing files 10_A2. Jon Hester - On The Move.mp3.wav with sample rate 16000
Processing files 90_together.mp3.wav with sample rate 16000
Processing files 330_Jamie XX - Let’s Do It Again (Marcelo Pantani Edit).mp3.wav with sample rate 16000
Processing files 290_Yamil, Stefano Crabuzza - Aint No Sunshine (Original Mix).mp3.wav with sample rate 16000
Processing files 340_Jon Hester - Arise.mp3.wav with sample rate 16000
Processing files 160_Orthoptera.mp3.wav with sample rate 16000
Processing files 70_B2 Jon Hester - Phantom Orbit.mp3.wav with sample rate 16000
Processing files 320_HUSP - The Flute - David Penn Remix.mp3.wav with s

Processing files 120_CV Premiere I Jon Hester - Sustain.mp3.wav with sample rate 16000
Processing files 70_Kind Of One - Bugs In Eden (Radio Edit).mp3.wav with sample rate 16000
Processing files 290_Dreamless.mp3.wav with sample rate 16000
Processing files 140_Mat.Joe – Nighthawk (Dosem Remix).mp3.wav with sample rate 16000
Processing files 90_Depth Of Silence.mp3.wav with sample rate 16000
Processing files 80_Jon Hester - Part 4.mp3.wav with sample rate 16000
Processing files 250_Analog People In A Digital World - Rose Rouge (Vocal Edit).mp3.wav with sample rate 16000
Processing files 280_Flashlight.mp3.wav with sample rate 16000
Processing files 1330_Mix 09:2013.mp3.mp3.wav with sample rate 16000
Processing files 100_01 Patrice Baumel - Vapour (Joris Voorn Solid Mix).mp3.wav with sample rate 16000
Processing files 180_Tomy Wahl, Danny Torres - Wait [Lemon Aid].mp3.wav with sample rate 16000
Processing files 260_Analog People In A Digital World - Rose Rouge (Vocal Edit).mp3.wav with s

Processing files 440_Juliet Fox & Derek Marin - Roomers (Alexander Aurel Remix).mp3.wav with sample rate 16000


In [46]:
faiss.write_index(index_wav_vec, '10sec_sample_index_wav2vec.index')
faiss.write_index(index_ast, '10sec_sample_index_wav2vec.index')

In [47]:
index_wav_vec.ntotal

10519

In [48]:
index_ast.ntotal

10519

## Testing FAISS similarity

In [ ]:
waveform, sample_rate = torchaudio.load('wav_files/10_seconds_samples/10_KW20 - Various Artists.mp3.wav')
waveform = waveform.squeeze().numpy()
display(Audio(waveform, rate=sample_rate))
print()
input_values = feature_extractor(waveform, return_tensors="pt", sampling_rate=16000).to('cuda:0')
embeddings_wav = model(**input_values).last_hidden_state
embeddings_wav = embeddings_wav.mean(dim=1)

input_values = feature_extractor_ast(waveform, return_tensors="pt", sampling_rate=16000).to('cuda:0')
embeddings_ast = model_ast(**input_values).last_hidden_state
embeddings_ast = embeddings_ast.mean(dim=1)

In [ ]:
D, I = index_wav_vec.search(embeddings_wav.to('cpu').detach().numpy(), k=10)
D_ast, I_ast = index_ast.search(embeddings_ast.to('cpu').detach().numpy(), k=10)

In [ ]:
print(D, D_ast)

In [ ]:
matched_file_wav = faiss_index_lookup_samples[I[0][1]]
matched_file_ast = faiss_index_lookup_samples[I_ast[0][1]]

In [ ]:
print(matched_file_wav, matched_file_ast)

In [ ]:
waveform, sample_rate = torchaudio.load('wav_files/10_seconds_samples/10_KW20 - Various Artists.mp3.wav')
waveform = waveform.squeeze().numpy()
display(Audio(waveform, rate=sample_rate))
print()

In [ ]:
print("Matched file wav2vec {}".format(matched_file_wav))
waveform, sample_rate = torchaudio.load(matched_file_wav)
waveform = waveform.squeeze().numpy()
display(Audio(waveform, rate=sample_rate))
print()

In [ ]:
print("Matched file wav2vec {}".format(matched_file_ast))

waveform, sample_rate = torchaudio.load(matched_file_ast)
waveform = waveform.squeeze().numpy()
display(Audio(waveform, rate=sample_rate))
print()

## Visualising the 2 the indices, creating a dataset with Huggingface Datasets

In [8]:
index_wav_vec = faiss.read_index('10sec_sample_index_wav2vec.index')
index_ast = faiss.read_index('10sec_sample_index_wav2vec.index')

In [59]:
import numpy as np
from docarray import DocumentArray
from docarray import base

docs = DocumentArray.empty(index_ast.ntotal)

In [ ]:
docs.embeddings

In [50]:
docs.embeddings = embeddings_ast

In [58]:
docs.plot_embeddings(port=8892, host='0.0.0.0')

/home/ubuntu/audio_llm/venv/lib/python3.10/site-packages/docarray/array/mixins/plot.py:277: UserWarning: Showing iframe in cell, you may want to open http://0.0.0.0:8892/static/index.html?config=config.json in a new tab for better experience. Also, `localhost` may need to be changed to the IP address if your jupyter is running remotely. Click "stop" button in the toolbar to move to the next cell.
  warnings.warn(


KeyboardInterrupt: 